In [168]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import ManagedOnlineDeployment, Environment, CodeConfiguration

#Creating handle to our service, replace with your own credentials
ml_client = MLClient(
    credential=DefaultAzureCredential(),
    subscription_id="ddfbea37-0529-4562-ac51-2ebbbe770146",
    resource_group_name="cloudServicesS242",
    resource_group = "cloudServicesS242",
    workspace_name="myWorkspace"
)

Creating MLClient won't connect to the workspace. The client initialization is lazy and waits for the first time it needs to make a call (this happens in the next code cell).

In [184]:
#Register model
# Import the necessary libraries
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

# Provide the model details, including the
# path to the model files, if you've stored them locally.
ml_model = Model(
    path="./app/my_model.keras",
    type=AssetTypes.CUSTOM_MODEL,
    name="my_model_keras",
    description="ML Model created from local files.",
)

# Register the model
ml_client.models.create_or_update(ml_model)



Model({'job_name': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'my_model_keras', 'description': 'ML Model created from local files.', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/ddfbea37-0529-4562-ac51-2ebbbe770146/resourceGroups/cloudServicesS242/providers/Microsoft.MachineLearningServices/workspaces/myWorkspace/models/my_model_keras/versions/4', 'Resource__source_path': '', 'base_path': 'c:\\Users\\vimajava\\Documents\\Tensorflow Azure', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x0000017B02B60BC0>, 'serialize': <msrest.serialization.Serializer object at 0x0000017B04A26F00>, 'version': '4', 'latest_version': None, 'path': 'azureml://subscriptions/ddfbea37-0529-4562-ac51-2ebbbe770146/resourceGroups/cloudServicesS242/workspaces/myWorkspace/datastores/workspaceblobstore/paths/LocalUpload/5da9b8e6524b0d09c5a4af0ade4cc029/my_model.keras', 

In [185]:
#confirm that ML model version is in Azure
registered_model_name = "my_model_keras"

# Let's pick the latest version of the model
latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name=registered_model_name)]
)

print(latest_model_version)

4


After you train a machine learning model, you need to deploy it so that others can use it for inferencing. For this purpose, Azure Machine Learning allows you to create endpoints and add deployments to them.

An endpoint, in this context, is an HTTPS path that provides an interface for clients to send requests (input data) to a trained model and receive the inferencing (scoring) results from the model. An endpoint provides:

Authentication using "key or token" based auth
TLS(SSL) termination
A stable scoring URI (endpoint-name.region.inference.ml.azure.com)
A deployment is a set of resources required for hosting the model that does the actual inferencing.

A single endpoint can contain multiple deployments. Endpoints and deployments are independent Azure Resource Manager resources that appear in the Azure portal.

Create online endpoint. Naming rules: https://learn.microsoft.com/en-us/azure/machine-learning/how-to-manage-quotas?view=azureml-api-2#azure-machine-learning-online-endpoints-and-batch-endpoints

get your subscription Id: az account list --output table
get your identity Id: az ad user show --id <your-email-or-username>
assign role for you: az role assignment create --assignee <identityId> --role "AzureML Data Scientist" --scope /subscriptions/<subscriptionId>/resourcegroups/<resourceGroupName>/providers/Microsoft.MachineLearningServices/workspaces/<workspaceName>

In [179]:
from azure.ai.ml.entities import ManagedOnlineEndpoint

online_endpoint_name = "myendpoint-ville"
# define an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is an online endpoint",
    auth_mode="key",
    tags={
        "training_dataset": "credit_defaults",
    },
)

In [172]:
# create the online endpoint
# expect the endpoint to take approximately 2 minutes.

endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

In [173]:
#get the endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpoint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

Endpoint "myendpoint-ville" with provisioning state "Succeeded" is retrieved


The key aspects of a deployment include:

name - Name of the deployment.

endpoint_name - Name of the endpoint that will contain the deployment.

model - The model to use for the deployment. This value can be either a reference to an existing versioned model in the workspace or an inline model specification.

environment - The environment to use for the deployment (or to run the model). This value can be either a reference to an existing versioned environment in the workspace or an inline environment specification. The environment can be a Docker image with Conda dependencies or a Dockerfile.

code_configuration - the configuration for the source code and scoring script.

path- Path to the source code directory for scoring the model.

scoring_script - Relative path to the scoring file in the source code directory. This script executes the model on a given input request. For an example of a scoring script, see Understand the scoring script in the "Deploy an ML model with an online endpoint" article.

instance_type - The VM size to use for the deployment. For the list of supported sizes, see Managed online endpoints SKU list.

instance_count - The number of instances to use for the deployment.

In [174]:
# Define the environment
env = Environment(
    name="my-custom-env",
    description="My custom environment with required packages",
    conda_file="conda.yml" , # Path to your conda file or requirements
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04")


In [193]:
from azure.ai.ml.entities import ManagedOnlineDeployment

# Choose the latest version of the registered model for deployment
model = ml_client.models.get(name=registered_model_name, version=latest_model_version)

# define an online endpoint
deployment = ManagedOnlineDeployment(
    name="mnist-deployment-v1",
    endpoint_name="myendpoint-ville",
    model=model,
    #environment="tensorflow-env:1",
    code_configuration=CodeConfiguration(
        code="./app", scoring_script="score.py"
        ),
    environment=env,
    
    instance_type="Standard_D2as_v4",
    instance_count=1
)

In [194]:
# create the online deployment
deployment = ml_client.online_deployments.begin_create_or_update(deployment, local=False).result()

# the deployment takes 100% traffic
# expect the deployment to take approximately 8 to 10 minutes.
endpoint.traffic = {"mnist-deployment-v1": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

Check: endpoint myendpoint-ville exists


Uploading app (2.49 MBs): 100%|##########| 2487290/2487290 [00:00<00:00, 3680937.47it/s]




..........................................

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://myendpoint-ville.northeurope.inference.ml.azure.com/score', 'openapi_uri': 'https://myendpoint-ville.northeurope.inference.ml.azure.com/swagger.json', 'name': 'myendpoint-ville', 'description': 'this is an online endpoint', 'tags': {'training_dataset': 'credit_defaults'}, 'properties': {'createdBy': 'Ville Majava', 'createdAt': '2024-10-01T05:14:03.150500+0000', 'lastModifiedAt': '2024-10-01T05:14:03.150500+0000', 'azureml.onlineendpointid': '/subscriptions/ddfbea37-0529-4562-ac51-2ebbbe770146/resourcegroups/cloudservicess242/providers/microsoft.machinelearningservices/workspaces/myworkspace/onlineendpoints/myendpoint-ville', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/ddfbea37-0529-4562-ac51-2ebbbe770146/providers/Microsoft.MachineLearningServices/locations/northeurope/mfeOperationsStatus/oeidp:ba617bca-4135-47aa-ac8b-e9574ba022b8:a7b10b63-c4e